In [173]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Importar otras librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------

import csv

# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None) # Muestra todas las columnas de los DataFrame
pd.set_option('display.width', None) # Ajusta el ancho de la consola para evitar truncamientos

1. Creamos conexión con SQL.

In [174]:
config = {
  'user': 'root',
  'password': 'AlumnaAdalab',
  'host': '127.0.0.1',
  'raise_on_warnings': True
}

try:
  cnx = mysql.connector.connect(**config)
  print("Conexión exitosa")

except mysql.connector.Error as err:

  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Hay un problema con tu nombre de usuario o contraseña")
  
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("La base de datos no existe")
  
  else:
    print(err)

# Tratamos de crear la conexión con SQL. Usamos try/except para manejar los posibles errores.

config = {
  'user': 'root',
  'password': 'AlumnaAdalab',
  'host': '127.0.0.1',
  'raise_on_warnings': True
}

try:
  cnx = mysql.connector.connect(**config)

except mysql.connector.Error as err:

  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Hay un problema con tu nombre de usuario o contraseña")
  
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("La base de datos no existe")
  
  else:
    print(err)

Conexión exitosa


2. Creamos nuestra base de datos: tocayoga

In [175]:
# Creamos un cursor para poder ejecutar nuestras consultas 

mycursor = cnx.cursor()

# Creamos nuestra base de datos (schema)

try:
    mycursor.execute("CREATE DATABASE tocayoga")
    print(mycursor)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)



CMySQLCursor: CREATE DATABASE tocayoga


Creamos una función para conectarnos con nuestra base de datos

In [176]:

def establecer_conexion(database):

    # Esta función conecta con la base de datos que se le indica por parámetro y devuelve el conector creado

    config = {
    'user': 'root',
    'password': 'AlumnaAdalab',
    'host': '127.0.0.1',
    'database' : f'{database}',
    'raise_on_warnings': True
    }

    try:
        cnx = mysql.connector.connect(**config)
        print(f"Conexión establecida con {database}")

    except mysql.connector.Error as err:

        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Hay un problema con tu nombre de usuario o contraseña")
        
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("La base de datos no existe")
        
        else:
            print(err)
    
    return cnx



Creamos la tabla categoria

In [177]:

cnx = establecer_conexion('tocayoga')

query_crear_tabla_categoria = """CREATE TABLE categoria (

    id_cat INT PRIMARY KEY,   
    nombre VARCHAR(20),
    beneficios VARCHAR(1000))
    """       

mycursor = cnx.cursor()

try:
    mycursor.execute(query_crear_tabla_categoria)
    print(mycursor)
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla.")
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)
               

Conexión establecida con tocayoga
CMySQLCursor: CREATE TABLE categoria (

    id_cat INT..


Creamos la tabla NIVEL

In [178]:

cnx = establecer_conexion('tocayoga')

query_crear_tabla_nivel = """CREATE TABLE nivel (
    id_nivel INT PRIMARY KEY,   
    nivel VARCHAR(20))
    """       

mycursor = cnx.cursor()

try:
    mycursor.execute(query_crear_tabla_nivel)
    print(mycursor)
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla.")
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)
               

Conexión establecida con tocayoga
CMySQLCursor: CREATE TABLE nivel (
    id_nivel INT PR..


Creamos la tabla POSTURA

TABLA POSTURA

(( id_postura PK 
id_cat FK 
id_nivel FK)) PK COMPUESTA

nombre
nombre_sans
descripcion
beneficios
url_svg,
url_png,
url_svg_alt

In [179]:

cnx = establecer_conexion('tocayoga')

query_crear_tabla_postura = """CREATE TABLE postura (

    id_postura INT,  
    id_cat INT, 
    id_nivel INT,
    nombre VARCHAR(100),
    nombre_sans VARCHAR(100),
    descripcion VARCHAR(2000),
    beneficios VARCHAR(2000),
    svg VARCHAR(500),
    png VARCHAR(500),
    alt VARCHAR(500),
    PRIMARY KEY (id_postura, id_cat, id_nivel),
    FOREIGN KEY (id_cat) REFERENCES categoria (id_cat),
    FOREIGN KEY (id_nivel) REFERENCES nivel (id_nivel))
    """       

mycursor = cnx.cursor()

try:
    mycursor.execute(query_crear_tabla_postura)
    print(mycursor)
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla.")
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

Conexión establecida con tocayoga
CMySQLCursor: CREATE TABLE postura (

    id_postura I..


Cargamos los df con los datos de los csv y preparamos las listas con los valores que irán en las tablas.

In [180]:
df_categoria = pd.read_csv('categorias.csv')
data_categoria = df_categoria.values.tolist()


In [181]:
df_categoria

,id,name,descripcion
0,1,Core Yoga,Engage your abdominal muscles with core yoga p...
1,2,Seated Yoga,Yoga practice with seated poses that help you...
2,3,Strengthening Yoga,Work and tone your entire body with strengthen...
3,4,Chest Opening Yoga,Open your heart and shoulders in chest opening...
4,5,Backbend Yoga,Discover the powerful effects of yoga backbend...
5,6,Forward Bend Yoga,"Learn how to work stiff muscles safely, promot..."
6,7,Hip Opening Yoga,"Loosen tight hips, improve your range of motio..."
7,8,Standing Yoga,Develop strength and stability in your standin...
8,9,Restorative Yoga,Restorative yoga focuses on winding down after...
9,10,Arm Balance Yoga,"Move past fear, build better balance, and stre..."


In [182]:
df_nivel = pd.read_csv('niveles.csv', index_col=0)
data_nivel = df_nivel.values.tolist()

In [183]:
df_nivel

,id_nivel,nivel
0,0,beginner
1,1,intermediate
2,2,expert


In [184]:
df = pd.read_csv('info_posturas_completo.csv', index_col=0)
df_postura = df[['id_postura','id_categoria', 'id_nivel', 'nombre', 'nombre_sans', 'descripcion',  'beneficios', 'url_svg', 'url_png', 'url_svg_alt']]
data_postura = df_postura.values.tolist()

Insertamos los valores en las tablas. 

In [185]:

#Insertar datos dentro de la tabla categoría

sql_categoria =  "INSERT INTO categoria (id_cat, nombre, beneficios) VALUES (%s,%s,%s)"

try: 
    mycursor.executemany(sql_categoria, data_categoria)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)


12 registro/s insertado/s.


In [186]:
data_nivel

[[0, 'beginner'], [1, 'intermediate'], [2, 'expert']]

In [187]:
#Insertar datos dentro de la tabla nivel

sql_nivel =  "INSERT INTO nivel (id_nivel, nivel) VALUES (%s, %s)"

try: 
    mycursor.executemany(sql_nivel, data_nivel)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)


3 registro/s insertado/s.


In [188]:
df.iloc[48]

id_postura                                                     10
nombre                                               Child's Pose
nombre_sans                                              Balāsana
id_categoria                                                    6
descripcion     From a kneeling position, the toes and knees a...
url_svg         https://res.cloudinary.com/dko1be2jy/image/upl...
url_png         https://res.cloudinary.com/dko1be2jy/image/upl...
url_svg_alt     https://www.dropbox.com/s/ini3uwali0q5gxa/chil...
nombre_cat                                      Forward Bend Yoga
beneficios      Learn how to work stiff muscles safely, promot...
nivel                                                    beginner
id_nivel                                                        0
Name: 48, dtype: object

In [189]:
#Insertar datos dentro de la tabla postura

# df_postura = df[['id_postura','id_categoria', 'nombre', 'nombre_sans', 'descripcion',  'beneficios','id_nivel', 'url_svg', 'url_png', 'url_svg_alt']]


sql_postura =  "INSERT INTO postura (id_postura, id_cat, id_nivel, nombre, nombre_sans, descripcion, beneficios, svg, png, alt) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

try: 
    mycursor.executemany(sql_postura, data_postura)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)



102 registro/s insertado/s.
